### Clear memory

In [ ]:
%reset -f
import gc
gc.collect()

0

### Import

In [ ]:
import pandas as pd
from pydantic import BaseModel, Field
import uvicorn, nest_asyncio
from fastapi import FastAPI, HTTPException, UploadFile, File, Form
from fastapi.middleware.cors import CORSMiddleware
from typing import List, Tuple, Dict, Optional, Any, Union
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)